In [ ]:
import sys
sys.path.append("../input/timmmaster")
sys.path.append("../input/pytorch-toolbelt")

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

from argparse import Namespace
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from torch.cuda.amp import autocast

import timm
from pytorch_toolbelt.inference import tta
from albumentations.pytorch import ToTensorV2
import cv2
import albumentations as A

from pytorch_lightning.utilities.seed import seed_everything

from pytorch_lightning import LightningDataModule, LightningModule
import pytorch_lightning as pl


In [ ]:
test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')

In [ ]:
class Pawpularity(pl.LightningModule):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.backbone = timm.create_model(config.backbone_name,
                                          pretrained=not config.predict,
                                          num_classes=0)
        self.head = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(self.backbone.num_features, 256),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(256, 1)
        )

        self.save_hyperparameters(config)

    def forward(self, x):
        return torch.sigmoid(self.head(self.backbone(x)))*100

In [ ]:
class PawpularityDataset(Dataset):
    def __init__(self, df, image_root, transform):
        self._X = df["Id"].values
        self._y = None
        if "Pawpularity" in df.keys():
            self._y = df["Pawpularity"].values
        self.transform = transform
        self.image_root = image_root

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = cv2.imread(self.image_root+image_path+'.jpg')
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        transformed = self.transform(image=image)
        transformed_image = transformed["image"]
        if self._y is not None:
            label = self._y[idx]
            return transformed_image, torch.FloatTensor([label])
        return transformed_image

In [ ]:
fold_preds = []

In [ ]:
def build_transform(x):
    pass

In [ ]:
def predict(config, weight=1):
    _, valid_transform = build_transform(config)
    print(valid_transform)
    for fold in range(5):
        print(fold)
        dataset = PawpularityDataset(test_df, config.img_root, valid_transform)
        dataloader = DataLoader(dataset, batch_size=16, shuffle=False, num_workers=2)

        model = Pawpularity.load_from_checkpoint(f'{config.path}{fold}.ckpt', 
                                                 config=config, strict=False)
        model.eval()
        model.cuda()

        all_pred, all_label = [], []
        with torch.no_grad():
            with autocast():
                for images in dataloader:
                    pred = tta.fliplr_image2label(model, images.cuda())
                    all_pred.append(pred.flatten().detach().cpu().numpy())

        all_pred = np.concatenate(all_pred)

        for _ in range(weight):
            fold_preds.append(all_pred)

## Seed 1991

In [ ]:
def build_transform(config):
    
    valid_aug = [
        A.ShiftScaleRotate(rotate_limit=15, p=0.5,border_mode=cv2.BORDER_CONSTANT, value=0,shift_limit=0),
#         A.SmallestMaxSize(max_size=config.img_size),
        A.RandomResizedCrop(config.img_size, config.img_size,scale=(0.7,1)),
        A.ColorJitter(p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ]
    valid_transform = A.Compose(valid_aug)
    return None, valid_transform

In [ ]:
seed_everything(1991)

In [ ]:
swin_base_384_config = Namespace(
    backbone_name = 'swin_base_patch4_window12_384_in22k',
    predict= True,
    img_root='../input/petfinder-pawpularity-score/test/',
    path = '../input/swin-base-bce-384-1225/swin_base_patch4_window12_384_in22k_',
    img_size=384
)
predict(swin_base_384_config, 1)

In [ ]:
cait_384_config = Namespace(
    backbone_name = 'cait_s36_384',
    predict= True,
    img_root='../input/petfinder-pawpularity-score/test/',
    path = '../input/cait-s-bce-384-1227/cait_s36_384_',
    img_size=384
)
predict(cait_384_config, 1)

In [ ]:
# xcit
config = Namespace(
    backbone_name = 'xcit_medium_24_p16_384_dist',
    predict= True,
    img_root='../input/petfinder-pawpularity-score/test/',
    path = '../input/xcit-m-bce-384-1228/xcit_medium_24_p16_384_dist_',
    img_size=384
)
predict(config, 1)

## seed 42

In [ ]:
def build_transform(config):
    valid_aug = [
        A.Resize(config.img_size, config.img_size),
        A.ColorJitter(p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ]
    valid_transform = A.Compose(valid_aug)
    return None, valid_transform

In [ ]:
seed_everything(42)

In [ ]:
config = Namespace(
    backbone_name = 'swin_base_patch4_window12_384_in22k',
    predict= True,
    img_root='../input/petfinder-pawpularity-score/test/',
    path = '../input/swin-384-42-0110/swin_base_patch4_window12_384_in22k_',
    img_size=384
)
predict(config, 1)

In [ ]:
config = Namespace(
    backbone_name = 'cait_s36_384',
    predict= True,
    img_root='../input/petfinder-pawpularity-score/test/',
    path = '../input/cait-s-384-42-0110/cait_s36_384_',
    img_size=384
)
predict(config, 1)

In [ ]:
# xcit
config = Namespace(
    backbone_name = 'xcit_medium_24_p16_384_dist',
    predict= True,
    img_root='../input/petfinder-pawpularity-score/test/',
    path = '../input/xcit-384-42-0110/xcit_medium_24_p16_384_dist_',
    img_size=384
)
predict(config, 1)

## seed 1019

In [ ]:
def build_transform(config):
    
    valid_aug = [
        A.ShiftScaleRotate(rotate_limit=15, p=0.5,border_mode=cv2.BORDER_CONSTANT, value=0,shift_limit=0),
#         A.SmallestMaxSize(max_size=config.img_size),
#         A.CenterCrop(config.img_size, config.img_size),
        A.RandomResizedCrop(config.img_size, config.img_size,scale=(0.7,1),ratio=(0.8,1.2)),
        A.ColorJitter(p=0.5),
        A.RandomBrightnessContrast(p=0.3),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ToTensorV2()
    ]
    valid_transform = A.Compose(valid_aug)
    return None, valid_transform

In [ ]:
seed_everything(1019)

In [ ]:
config = Namespace(
    backbone_name = 'swin_large_patch4_window12_384_in22k',
    predict= True,
    img_root='../input/petfinder-pawpularity-score/test/',
    path = '../input/slarge-384-1019-0112/swin_large_patch4_window12_384_in22k_',
    img_size=384
)
predict(config, 2)

In [ ]:
config = Namespace(
    backbone_name = 'swin_large_patch4_window7_224_in22k',
    predict= True,
    img_root='../input/petfinder-pawpularity-score/test/',
    path = '../input/swin-large-224-1019-0110/swin_large_patch4_window7_224_in22k_',
    img_size=224
)
predict(config, 1)

In [ ]:
print(len(fold_preds))

In [ ]:
test_df['Pawpularity'] = np.mean(fold_preds, axis=0)

In [ ]:
test_df[['Id', 'Pawpularity']].to_csv('submission.csv', index=False)

In [ ]:
test_df[['Id', 'Pawpularity']].head(10)